In [75]:
import importlib as imp
import functions as f
imp.reload(f)
import dictionaries as di
from functions import load_songs_from_xml, random_sample_df, reverse_dictionary, calculate_new_tempo, new_nml_key_calculator, nml_key_int_to_name, row_skipper, BPM_skipper
from dictionaries import harmonic_to_note
import importlib as imp
note_to_harmonic = reverse_dictionary(harmonic_to_note)
from functions import load_songs_from_xml, random_sample_df, reverse_dictionary
import pandas as pd

import paralell_graph
from paralell_graph import parallel_job

In [76]:
from functions import traktor_int_to_note, musical_key_to_harmonic_key

In [77]:

file = "BreakBeat not listened.nml"
file = "Breakbeat2023-09.nml"
file = "temp.nml"
file = "NewBreakBeatMixList.nml"
#file = "BREAK_ALL.nml"
#file = "TestPlaylist.nml"
# file = "BreakBeatTest.nml"

df = load_songs_from_xml(file)
# schuffle df
df = df.sample(frac=1).reset_index(drop=True)
sampled_df = random_sample_df(df, n_samples=0)

Number of songs with missing Artist or Title: 55, number of songs handled: 110


In [78]:
# List to hold the new rows
new_rows = []

SEMITONES = [-4, -3, -2, -1, 0, 1, 2,3, 4, 5, 6]
SEMITONES = [-2, -1, 0, 1, 2]
# SEMITONES = [-3, 0]
bpm_semitone_threshold = 0.25

#### Mi van a  nullával? 
# SEMITONES = [-2, -1, 1, 2]

# Replace 'sampled_df' with your actual DataFrame variable name
for row in sampled_df.itertuples(index=False):  # Assuming you don't need the DataFrame index
    # Check if the row contains the '-r' comment
    if row.Musical_Key_Int < 12:
        min_or_maj = "major"
    else:
        min_or_maj = "minor"
    if row_skipper(row) and BPM_skipper(row, row.BPM):
        for semitone_change in SEMITONES:
            new_tempo = calculate_new_tempo(row.BPM, semitone_change)
            #####EZ ITT MÉG MINDIG KURVÁRA NEM JÓ
            buzi_calc = (row.Musical_Key_Int + semitone_change) % 24
            new_key_int = f.new_nml_key_calculator(min_or_maj, buzi_calc)
            musical_key = traktor_int_to_note[new_key_int][0]
            harmonic_key = musical_key_to_harmonic_key[musical_key]

            # Create a dictionary for the new row
            new_row = {
                "ID": row.ID,
                "Artist": row.Artist,
                "Title": row.Title,
                "File_Path": row.File_Path,
                "Play_Count": row.Play_Count,
                "Comment": row.Comment,
                "Color": row.Color,
                "BPM": new_tempo,
                "BPM_max": calculate_new_tempo(new_tempo, bpm_semitone_threshold),
                "BPM_min": calculate_new_tempo(new_tempo, -bpm_semitone_threshold),
                "Musical_Key_Int": new_key_int,
                "Musical_Key": musical_key,
                "Harmonic_Key": harmonic_key,
                "Kind": "Calculated",
            }

            # Add the new row to the list
            new_rows.append(new_row)
        else:
            None
    else:
        None

# Create a new DataFrame from the list of new rows
new_df = pd.DataFrame(new_rows)
df_combined = pd.concat([new_df], ignore_index=True)
# df_combined = new_df

### 🐌 Try to make a faster itertuples implementation

### 🏃‍♀️ Fast comparisson

In [8]:
imp.reload(f)

def fast_comparisson(df):
    df_list = df.values.tolist()
    edge_list_to_list = []
    for row_base in df_list:
        for row_to_compare in df_list:
            if row_base[0] == row_to_compare[0]:
                None
            else:
                # first filter, song is whennever between ranges?
                # 7 alap, 8 max, 9 min, 10 musical key init
                if row_base[7] >= row_to_compare[9] and row_base[7] <= row_to_compare[8]:
                    # only a temporary function:
                    if row_base[10] == row_to_compare[10]:
                        edge_list_to_list.append((row_base[0], row_to_compare[0]))
                        # I think it is a duplicated funciton 
                    #### BIG BIG TODO DOESNT WORK IN HERE!!!!!!!!!!!!!!
                    elif row_base[12] in f.get_classic_neighbours(row_to_compare[12]):
                        #print(row_base.ID, row_to_compare.ID)
                        edge_list_to_list.append((row_base[0], row_to_compare[0]))
                    else:
                        None
                else:
                    None
    return edge_list_to_list

# edge_list = fast_comparisson(df_combined)
edge_list = fast_comparisson(df_combined)


## 🚅 Fastest --- this is not fast at all

In [65]:
# edge_list = parallel_job(df_combined, cores=8)

In [71]:
import networkx as nx

# Assuming 'edge_list' is already defined as shown in your example

# Create a graph from the edge list
G = nx.DiGraph()
G.add_edges_from(edge_list)

# Now, you can access edges connected to a specific node
node_id = list(G.nodes())[15]  # Example node ID
connected_nodes = list(G.successors(node_id))

print(f"Nodes directly connected to node {node_id}: {connected_nodes}")

Nodes directly connected to node 58: [62, 42, 110, 47, 69, 51, 109, 64, 44, 73, 123]


In the context of directed graphs, such as the one represented by a DiGraph object in NetworkX, the "in-degree" of a node refers to the number of edges coming into that node from other nodes. More specifically, if you have a directed edge from node A to node B, this contributes to the in-degree of node B.

In [79]:
# Assuming G is your directed graph (DiGraph)
import random 

def random_unique_path(G, path_length):
    if len(G) < path_length:
        raise ValueError("Graph has fewer nodes than the desired path length.")
    
    # Start from a random node
    path = [random.choice(list(G.nodes()))]
    
    # Try to find a path with the specified length
    while len(path) < path_length:
        # Get the neighbors of the last node in the path
        neighbors = list(G.successors(path[-1]))
        # Filter out neighbors that are already in the path
        neighbors = [node for node in neighbors if node not in path]
        
        if not neighbors:
            # If there are no neighbors that haven't been visited, start over
            path = [random.choice(list(G.nodes()))]
        else:
            # Randomly select one of the unvisited neighbors
            path.append(random.choice(neighbors))
    
    return path

# Generate a random unique path of 20 nodes
path_length = 20
random_path = random_unique_path(f.keep_largest_cluster(G), path_length)

print("Random unique path:", random_path)

Random unique path: [1, 85, 36, 82, 101, 104, 10, 40, 34, 74, 5, 76, 91, 73, 97, 109, 125, 88, 32, 17]


In [80]:
# Specify the drive letter that is missing from your file paths
drive_letter = "E:"
from datetime import datetime
# Create the M3U file

now = str(datetime.now())
now = now.replace(":", "-")
with open(f"playlist_{now}.m3u", "w", encoding='utf-8') as m3u_file:
    for i in random_path:
        # Retrieve the file path for the given ID and prepend the drive letterth)
        file_path = df.loc[df.ID == i].File_Path.values[0]
        full_path = drive_letter + file_path.replace('//', '\\')
        print(full_path)
        # Write the full path to the M3U file
        print
        m3u_file.write(full_path + "\n")

print("M3U playlist created as 'playlist.m3u'.")

E:\DJ\BREAKBEAT\Alarm Bells.mp3
E:\DJ\SELECTOR\Pardon Moi - Power to the People (Markus Gibb Remix)!HARD.mp3
E:\DJ\ALBUMOK\[torrents.ru]_breaks_pack_march_2009\VA-Breakspoll_2009-WEB-2009-B2R\09-stanton_warriors-precinct_(original_mix).mp3
E:\DJ\SELECTOR\Moisees - Midnight Raider (Original Mix).mp3
E:\DJ\SELECTOR\Tunnel Signs - Cracks (Emile Strunz Remix).mp3
E:\DJ\SELECTOR\Undefined Pattern - Estrex Humano.mp3
E:\DJ\BREAKBEAT\Lilly in the Sky.mp3
E:\DJ\CHILL\Nightmares On Wax - Feelin Good -(2013)-[320]-\Nightmares On Wax - Be, I Do.mp3
E:\DJ\ALBUMOK\[torrents.ru]_breaks_pack_march_2009\Mesmer-Karate_Kuts-(BTX0011)-WEB-2008-CBR\01-mesmer-karate_kuts_(original_mix).mp3
E:\DJ\SELECTOR\Lust & Fury Edits - Shout (Mufti Edit).mp3
E:\DJ\BREAKBEAT\Dont Stop.mp3
E:\DJ\SELECTOR\Meat Katie & Elite Force - The answer (Force Mass Motion Phunk-Acid mix).mp3
E:\DJ\SELECTOR\Rodion, Mammarella - Barranca Del Cobre [Slow Motion].mp3
E:\DJ\SELECTOR\Ludviq, Technicism - Metalizer.mp3
E:\DJ\SELECTOR\Stev

In [79]:
def get_key_number(key):
    return int(key[:-1])

def get_key_letter(key):
    return key[-1]

def get_weight(mix_type):
    weights = {
        'perfect_mix': 10,
        'minus_1_mix': 8,
        'plus_1_mix': 9,
        'scale_change': 7,
        'energy_boost': 6,
        'mood_shifter': 5
    }
    return weights.get(mix_type, 0)

def get_mix_type(base_key, compare_key):
    # Convert key codes to numbers and letters
    base_number = get_key_number(base_key)
    compare_number = get_key_number(compare_key)
    base_letter = get_key_letter(base_key)
    compare_letter = get_key_letter(compare_key)

    # Define transitions for different mix types
    transitions = {
        0: ['perfect_mix', 'scale_change'],
        1: ['minus_1_mix', 'plus_1_mix'],
        2: ['energy_boost'],
        3: ['scale_change'],

    }
    diff = abs(base_number - compare_number) % 12
    
    if diff > 6:
        diff = 12 - diff

    ### Perfect mix is tested, good
    if diff == 0 and base_letter == compare_letter:
        return 'perfect_mix'
    elif diff == 0 and base_letter != compare_letter:
        return 'scale_change'
    
    ### This is tested
    diff = (compare_number - base_number) % 12
    if diff == 11 and base_letter == compare_letter:  # 11 in modulo 12 arithmetic is effectively -1
        return 'minus_1_mix'
    elif diff == 1 and base_letter == compare_letter:
        return 'plus_1_mix'

    # Tested, working
    elif diff == 2 and base_letter == compare_letter:
        return 'energy_boost'
    
    # tested, working
    elif base_letter == 'b' and compare_letter == 'a':
        if (base_number - 3) % 12 == (compare_number) % 12:  # Adjust for 'b' to 'a' transition
            return 'mood_shifter'
        else:
            return 'unknown'
    elif base_letter == 'a' and compare_letter == 'b':
        if (base_number + 3) % 12 == (compare_number) % 12:  # Adjust for 'a' to 'b' transition
            return 'mood_shifter'
        else:
            return 'unknown'
    else:
        return 'unknown'

In [64]:
"""for b_n in range(1, 13):
    for b_l in ['a', 'b']:
        for c_n in range(1, 13):
            for c_l in ['a', 'b']:
                get_mix = get_mix_type(str(b_n) + b_l, str(c_n) + c_l)
                if get_mix == 'energy_boost':
                    print(f"{b_n}{b_l} to {c_n}{c_l}: {get_mix}")"""

'for b_n in range(1, 13):\n    for b_l in [\'a\', \'b\']:\n        for c_n in range(1, 13):\n            for c_l in [\'a\', \'b\']:\n                get_mix = get_mix_type(str(b_n) + b_l, str(c_n) + c_l)\n                if get_mix == \'energy_boost\':\n                    print(f"{b_n}{b_l} to {c_n}{c_l}: {get_mix}")'

In [ ]:
def new_fast_comparisson(df):
    df_list = df.values.tolist()
    edge_list_to_list = []
    edge_dict = {}
    for row_base in df_list:
        for row_to_compare in df_list:
            if row_base[0] == row_to_compare[0]:
                None
            else:
                # first filter, song is whennever between ranges?
                # 7 alap, 8 max, 9 min, 10 musical key init
                if row_base[7] >= row_to_compare[9] and row_base[7] <= row_to_compare[8]:
                    # only a temporary function:
                    get_mix = get_mix_type(row_base[12], row_to_compare[12])
                    if row_base[10] == row_to_compare[10]:
                        edge_list_to_list.append((row_base[0], row_to_compare[0]))
                        # I think it is a duplicated funciton 
                    #### BIG BIG TODO DOESNT WORK IN HERE!!!!!!!!!!!!!!
                    elif get_mix != "unknown":
                        #print(row_base.ID, row_to_compare.ID)
                        # somehow I wanna append a dictionary. Like row_base[0][get_mix].append(row_to_compare[0])
                        edge_list_to_list.append((row_base[0], row_to_compare[0]))
                    else:
                        None
                else:
                    None
    return edge_list_to_list

# edge_list = fast_comparisson(df_combined)
edge_list = new_fast_comparisson(df_combined)

In [80]:
def new_fast_comparisson(df):
    df_list = df.values.tolist()
    edge_dict = {}  # Initialize a dictionary to hold the connections

    for row_base in df_list:
        base_id = row_base[0]  # Get the ID of the base song
        if base_id not in edge_dict:
            edge_dict[base_id] = {}  # Initialize a dictionary for this base song if it doesn't exist

        for row_to_compare in df_list:
            compare_id = row_to_compare[0]  # Get the ID of the comparison song
            if base_id != compare_id:  # Ensure we're not comparing a song with itself
                # Check if the comparison song is within the BPM range of the base song
                if row_base[7] >= row_to_compare[9] and row_base[7] <= row_to_compare[8]:
                    get_mix = get_mix_type(row_base[12], row_to_compare[12])  # Determine the mix type
                    if get_mix != "unknown":  # If a valid mix type is found
                        if get_mix not in edge_dict[base_id]:
                            edge_dict[base_id][get_mix] = []  # Initialize a list for this mix type if it doesn't exist
                        edge_dict[base_id][get_mix].append(compare_id)  # Append the comparison song ID to the list

    return edge_dict

In [81]:
edge_list = new_fast_comparisson(df_combined)

In [82]:
edge_list

{7: {'energy_boost': [40, 40, 40],
  'minus_1_mix': [30, 30, 32, 30, 32, 30, 32, 30, 32],
  'plus_1_mix': [2, 2, 2, 2, 2],
  'perfect_mix': [48, 48, 48, 48]},
 34: {'minus_1_mix': [14, 46, 14, 46, 14, 46, 14],
  'perfect_mix': [33, 39, 41, 33, 39, 41, 33, 39, 41, 33, 39, 41, 33, 39, 41],
  'energy_boost': [49, 49, 49, 49, 49],
  'scale_change': [29, 29, 29, 29, 29],
  'plus_1_mix': [52, 52, 52, 52]},
 51: {'energy_boost': [34,
   33,
   39,
   41,
   34,
   33,
   39,
   41,
   34,
   33,
   39,
   41,
   34,
   33,
   39,
   41],
  'minus_1_mix': [10, 25, 26, 10, 25, 26, 10, 25, 26, 10, 25, 26]},
 31: {'perfect_mix': [8, 48, 4, 8, 48, 4, 8, 48, 4, 8, 48, 4, 8, 48, 4],
  'minus_1_mix': [45, 45, 45]},
 40: {'scale_change': [37, 37, 37, 37, 37],
  'energy_boost': [36, 36, 36],
  'minus_1_mix': [2, 2, 2],
  'plus_1_mix': [22, 20, 28, 35, 22, 20, 28, 35, 22, 20]},
 11: {'energy_boost': [31,
   8,
   48,
   4,
   31,
   8,
   48,
   4,
   31,
   8,
   48,
   4,
   31,
   8,
   48,
   4,
   

In [85]:
df_combined[df_combined.ID == 34]

,ID,Artist,Title,File_Path,Play_Count,Comment,Color,BPM,BPM_max,BPM_min,Musical_Key_Int,Musical_Key,Harmonic_Key,Kind
5,34,Atlantis,Fiji (Mesmer Remix),//DJ//SELECTOR//Atlantis_-_Fiji_(Mesmer_Remix)...,3,Free download www.soundcloud.com/mesmer,6,118.489530,120.212999,116.790769,13,C#m,12A,Calculated
6,34,Atlantis,Fiji (Mesmer Remix),//DJ//SELECTOR//Atlantis_-_Fiji_(Mesmer_Remix)...,3,Free download www.soundcloud.com/mesmer,6,125.535284,127.361236,123.735509,14,Dm,7A,Calculated
7,34,Atlantis,Fiji (Mesmer Remix),//DJ//SELECTOR//Atlantis_-_Fiji_(Mesmer_Remix)...,3,Free download www.soundcloud.com/mesmer,6,133.000000,134.934530,131.093205,15,D#m,2A,Calculated
8,34,Atlantis,Fiji (Mesmer Remix),//DJ//SELECTOR//Atlantis_-_Fiji_(Mesmer_Remix)...,3,Free download www.soundcloud.com/mesmer,6,140.908592,142.958154,138.888413,16,Em,9A,Calculated
9,34,Atlantis,Fiji (Mesmer Remix),//DJ//SELECTOR//Atlantis_-_Fiji_(Mesmer_Remix)...,3,Free download www.soundcloud.com/mesmer,6,149.287452,151.458888,147.147148,17,Fm,4A,Calculated


In [86]:
df_combined[df_combined.ID == 49]

,ID,Artist,Title,File_Path,Play_Count,Comment,Color,BPM,BPM_max,BPM_min,Musical_Key_Int,Musical_Key,Harmonic_Key,Kind
80,49,Llupa,Ignition,//DJ//Llupa - Ignition.mp3,1,None,,118.489530,120.212999,116.790769,15,D#m,2A,Calculated
81,49,Llupa,Ignition,//DJ//Llupa - Ignition.mp3,1,None,,125.535284,127.361236,123.735509,16,Em,9A,Calculated
82,49,Llupa,Ignition,//DJ//Llupa - Ignition.mp3,1,None,,133.000000,134.934530,131.093205,17,Fm,4A,Calculated
83,49,Llupa,Ignition,//DJ//Llupa - Ignition.mp3,1,None,,140.908592,142.958154,138.888413,18,F#m,11A,Calculated
84,49,Llupa,Ignition,//DJ//Llupa - Ignition.mp3,1,None,,149.287452,151.458888,147.147148,19,Gm,6A,Calculated


In [72]:
for i in range (0,24):
    print(f"{i}: {traktor_int_to_note[i]}")

0: ['C']
1: ['C#', 'Db']
2: ['D']
3: ['D#', 'Eb']
4: ['E']
5: ['F']
6: ['F#', 'Gb']
7: ['G']
8: ['G#', 'Ab']
9: ['A']
10: ['A#', 'Bb']
11: ['B']
12: ['Cm']
13: ['C#m', 'Dbm']
14: ['Dm']
15: ['D#m', 'Ebm']
16: ['Em']
17: ['Fm']
18: ['F#m', 'Gbm']
19: ['Gm']
20: ['G#m', 'Abm']
21: ['Am']
22: ['A#m', 'Bbm']
23: ['Bm']
